# What Office Character Do You Talk Like?



We are going to download all of The Office scripts and put them in a python dataframe. 

* Find source data of scripts
* pull into python
* transform the data into easy to work with dataframes
* run sentiment analysis on each character's line
* create a function asking for end users input


<hr size="5"/>

### Table of Contents
* [1. Finding Script Data](#Finding-Script-Data)
* [2. Data Manipulation](#Fixing-Inconsistancies)
* [3. Sentiment](#Sentiment)

# Finding the Text to Every Episode's Script

There is a Google docs sheet that has exactly what we are looking for located [here](https://docs.google.com/spreadsheets/d/18wS5AAwOh8QO95RwHLS95POmSNKA2jjzdt0phrxeAE0/edit#gid=747974534). We should download and save thsi to our desktop. We can pull the dataframe into python by using `pandas`. 

In [2]:
import pandas as pd 

the_office_raw_script = pd.read_excel(r"C:\Users\JF\Desktop\the-office-lines.xlsx")

Some general tips for looking at a dataframe. 
* How many rows does it have?
* What are the column names?
* Group by count.


In [3]:
# How many rows?
len(the_office_raw_script.index)

59909

59,909 lines of dialog happened in The Office!

In [4]:
# Column names?
list(the_office_raw_script)

['id',
 'season',
 'episode',
 'scene',
 'line_text',
 'speaker',
 'deleted',
 'Stage1']

We have eight columns, looks like `speaker` is the column we are going to be most interested in. 

In [5]:
# How many speakers have over 100 scenes?
s = the_office_raw_script.groupby('speaker').scene.count()
# s[s > 100].index.tolist()

# Fixing Inconsistancies

* Fix capalization
* Fix spelling



In [6]:
#Capalization
the_office_raw_script['speaker'] = the_office_raw_script['speaker'].str.lower()

In [7]:
# Fix Spelling
the_office_raw_script['speaker']  = the_office_raw_script['speaker'].replace(['micheal', 'michel', 'michal', 'Michael [on phone]'], 'michael')

# Sentiment
* import `TextBlob`

In [8]:
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
analyser = SentimentIntensityAnalyzer()

In [9]:
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)

In [10]:
the_office_raw_script['subjectivity'] = the_office_raw_script['line_text'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

the_office_raw_script['polarity'] = the_office_raw_script['line_text'].apply(lambda x: TextBlob(x).sentiment.polarity)

the_office_raw_script['scores'] = the_office_raw_script['line_text'].apply(lambda x: analyser.polarity_scores(x))



In [12]:
# using the following link to transform [scores] to four columns 
# https://stackoverflow.com/questions/50512188/unpack-dictionary-from-pandas-column
scores = the_office_raw_script['scores']
scores_spread = pd.DataFrame.from_records(scores.tolist())

In [13]:
the_office_raw_script = pd.concat([scores_spread, the_office_raw_script], axis=1)

In [14]:
the_office_raw_script.head()

,compound,neg,neu,pos,id,season,episode,scene,line_text,speaker,deleted,Stage1,subjectivity,polarity,scores
0,0.4927,0.0,0.803,0.197,1,1,1,1,All right Jim. Your quarterlies look very good...,michael,False,Michael,0.657857,0.597857,"{'neg': 0.0, 'neu': 0.803, 'pos': 0.197, 'comp..."
1,0.0000,0.0,1.000,0.000,2,1,1,1,"Oh, I told you. I couldn't close it. So...",jim,False,not Michael,0.000000,0.000000,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,0.0000,0.0,1.000,0.000,3,1,1,1,So you've come to the master for guidance? Is ...,michael,False,Michael,0.000000,0.000000,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,0.4215,0.0,0.714,0.286,4,1,1,1,"Actually, you called me in here, but yeah.",jim,False,not Michael,0.100000,0.000000,"{'neg': 0.0, 'neu': 0.714, 'pos': 0.286, 'comp..."
4,0.2732,0.0,0.811,0.189,5,1,1,1,"All right. Well, let me show you how it's done.",michael,False,Michael,0.535714,0.285714,"{'neg': 0.0, 'neu': 0.811, 'pos': 0.189, 'comp..."


we have all of the sentiment above, for a better comparison of 

In [15]:
the_office_class = the_office_raw_script[ (the_office_raw_script['speaker'] == 'jim') |
                             (the_office_raw_script['speaker'] == 'michael') |
                           (the_office_raw_script['speaker'] == 'pam') |
                            (the_office_raw_script['speaker'] == 'andy') |
                            (the_office_raw_script['speaker'] == 'dwight')]

In [16]:
the_office_class['num_speaker'] = the_office_class.speaker.map({'jim':1, 'michael':2, 'pam':3, 'andy':4, 'dwight':5})

C:\Users\JF\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
speaker = the_office_class['num_speaker']
line_text = the_office_class['line_text']
from collections import Counter
print(Counter(speaker))

Counter({2: 12153, 5: 7529, 1: 6815, 3: 5375, 4: 3968})


In [18]:

from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Boosting the number of samples in the speakers with lower lines. 

In [20]:
#pam
the_office_class_pam = the_office_class[the_office_class['speaker'] == "pam"]
the_office_class_pam_resample = resample(the_office_class_pam, 
                                         replace=True,     # sample with replacement
                                         n_samples=12137,    # to match majority class
                                         random_state=123) # reproducible results
 
    
the_office_class_jim = the_office_class[the_office_class['speaker'] == "jim"]
the_office_class_jim_resample = resample(the_office_class_jim, 
                                         replace=True,     # sample with replacement
                                         n_samples=12137,    # to match majority class
                                         random_state=123) # reproducible results


the_office_class_dwight = the_office_class[the_office_class['speaker'] == "dwight"]
the_office_class_dwight_resample = resample(the_office_class_dwight, 
                                         replace=True,     # sample with replacement
                                         n_samples=12137,    # to match majority class
                                         random_state=123) # reproducible results

the_office_class_andy = the_office_class[the_office_class['speaker'] == "andy"]
the_office_class_andy_resample = resample(the_office_class_andy, 
                                         replace=True,     # sample with replacement
                                         n_samples=12137,    # to match majority class
                                         random_state=123) # reproducible results 


adding the speaker with the most lines back in

In [21]:
the_office_class_michael = the_office_class[the_office_class['speaker'] == "michael"]

In [22]:
df_upsampled = pd.concat([the_office_class_michael, 
                          the_office_class_andy_resample,
                          the_office_class_dwight_resample,
                          the_office_class_jim_resample,
                          the_office_class_pam_resample ])

In [23]:
df_upsampled.num_speaker.value_counts()

2    12153
5    12137
4    12137
3    12137
1    12137
Name: num_speaker, dtype: int64

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB


X_train, X_test, y_train, y_test = train_test_split(df_upsampled['line_text'], df_upsampled['speaker'], random_state = 0)

count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [25]:
print(clf.predict(count_vect.transform(["assistant to the reginal manager"])))

['dwight']


In [26]:
print(clf.predict(count_vect.transform(["that's what she said"])))

['pam']
